In [ ]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import subprocess
import gzip
import re
from Bio import SeqIO

In [ ]:
metazoa = 33208
bilateria = 33213
eukaryota = 2759

In [ ]:
# Function for parsing out EGGNOG orthogroups
def parseOG( ogStr, level ):
    ogs = ogStr.split( ',' )
    for og in ogs:
        if str(level) in og:
            return og.split('@')[0]

In [5]:
mlig = pd.read_table( 'Refs/Mlig_4_5.v5/EGGNOG/Mlig.emapper.annotations', skiprows=4, skipfooter=3 )
mlig = mlig.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
mlig['Species'] = 'Mlig'
mlig['Metazoa'] = mlig.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
mlig['Bilateria'] = mlig.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
mlig['Eukaryota'] = mlig.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
mlig

/tmp/ipykernel_32011/1336268425.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  mlig = pd.read_table( '../../Refs/Mlig_4_5.v5/EGGNOG/Mlig.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,Mlig455_000001,6.190000e-16,89.7,"KOG2177@1|root,KOG2177@2759|Eukaryota,39W1B@33...",33208|Metazoa,interferon-beta production,TRIM56,"GO:0000003,GO:0000209,GO:0001816,GO:0001817,GO...",2.3.2.27,"ko:K11997,ko:K12026","NHL,zf-B_box,zf-C3HC4,zf-RING_UBOX",Mlig,3BIP6,3CRTC,KOG2177
1,Mlig455_000002,3.180000e-89,300.0,"KOG3512@1|root,KOG3512@2759|Eukaryota,39ZRD@33...",33208|Metazoa,Laminin N-terminal domain (domain VI),NTN3,"GO:0000768,GO:0000902,GO:0000904,GO:0003674,GO...",-,ko:K06844,"Laminin_EGF,Laminin_N,NTR",Mlig,3BNCU,3D4V1,KOG3512
2,Mlig455_000003,5.890000e-07,59.3,"28J4C@1|root,2QRGC@2759|Eukaryota,39AM6@33154|...",33208|Metazoa,Coiled-coil domain containing 150,CCDC150,-,-,ko:K15201,-,Mlig,3BGX4,3CVC0,2QRGC
3,Mlig455_000004,3.240000e-67,226.0,"KOG1655@1|root,KOG1655@2759|Eukaryota,38RIY@33...",33208|Metazoa,vacuolar transport,CHMP5,"GO:0000070,GO:0000278,GO:0000280,GO:0000815,GO...",-,ko:K12198,Snf7,Mlig,3BEAB,None,KOG1655
4,Mlig455_000005,6.060000e-31,134.0,"COG1597@1|root,KOG1116@2759|Eukaryota,38DRX@33...",33208|Metazoa,Sphingosine kinase 2,SPHK2,"GO:0000003,GO:0000323,GO:0001505,GO:0001568,GO...",2.7.1.91,ko:K04718,DAGK_cat,Mlig,3BDM3,3CZEF,KOG1116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38162,Mlig455_071171,3.400000e-35,147.0,"KOG3804@1|root,KOG3804@2759|Eukaryota,39R90@33...",33208|Metazoa,Sequence-specific DNA binding. It is involved ...,ELF2,"GO:0000003,GO:0000981,GO:0000982,GO:0001077,GO...",-,"ko:K03211,ko:K09428","Elf-1_N,Ets",Mlig,3BA8E,3CYFC,KOG3804
38163,Mlig455_071170,5.140000e-33,121.0,"COG0537@1|root,KOG3275@2759|Eukaryota,3A3J2@33...",33208|Metazoa,HIT domain,HINT1,"GO:0000118,GO:0003674,GO:0003824,GO:0005080,GO...",-,ko:K02503,HIT,Mlig,3BRD5,3D8AA,KOG3275
38164,Mlig455_071168,9.890000e-160,518.0,"KOG2157@1|root,KOG2157@2759|Eukaryota,38EAN@33...",33208|Metazoa,"Tubulin tyrosine ligase-like family, member",TTLL8,"GO:0000003,GO:0000226,GO:0001578,GO:0003006,GO...",-,"ko:K05755,ko:K16608",TTL,Mlig,3B9RM,3CVG8,KOG2157
38165,Mlig455_071158,3.490000e-36,143.0,"COG1236@1|root,KOG1361@2759|Eukaryota,38HCF@33...",33208|Metazoa,DNA cross-link repair 1A,DCLRE1A,"GO:0000228,GO:0000715,GO:0000723,GO:0000726,GO...",-,ko:K15340,"DRMBL,Lactamase_B_2",Mlig,3BB6K,3D188,KOG1361


In [6]:
smed = pd.read_table( 'Refs/dd_Smes_g4/EGGNOG/Smed.emapper.annotations', skiprows=4, skipfooter=3 )
smed = smed.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
smed['Species'] = 'Smed'
smed['Metazoa'] = smed.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
smed['Bilateria'] = smed.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
smed['Eukaryota'] = smed.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
smed

/tmp/ipykernel_32011/964412979.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  smed = pd.read_table( '../../Refs/SMESG/EGGNOG/Smed.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,SMESG000000003.1,2.320000e-65,223.0,"COG0017@1|root,COG0017@2|Bacteria,4NDY4@976|Ba...",976|Bacteroidetes,Asparaginyl-tRNA synthetase,asnS,-,6.1.1.22,ko:K01893,"tRNA-synt_2,tRNA_anti-codon",Smed,None,None,None
1,SMESG000000013.1,1.130000e-127,432.0,"COG1057@1|root,COG5113@1|root,KOG4015@1|root,K...",33208|Metazoa,Ubiquitination factor E4B,UBE4B,"GO:0000151,GO:0000166,GO:0000209,GO:0003007,GO...",2.3.2.27,ko:K10597,"U-box,Ufd2P_core",Smed,3BAI3,3CTAM,KOG2042
2,SMESG000000022.1,8.780000e-10,67.4,"KOG0254@1|root,KOG0254@2759|Eukaryota,37MDA@33...",35493|Streptophyta,Belongs to the major facilitator superfamily. ...,-,"GO:0000325,GO:0003674,GO:0005215,GO:0005365,GO...",-,ko:K08150,Sugar_tr,Smed,None,None,KOG0254
3,SMESG000000023.1,1.990000e-11,68.2,"KOG4691@1|root,KOG4691@2759|Eukaryota,3A3EB@33...",33208|Metazoa,Mitochondrial ribosome subunit S26,mRpS26,"GO:0000313,GO:0000314,GO:0003674,GO:0003735,GO...",-,ko:K17405,MRP-S26,Smed,3BS0W,3D8DT,KOG4691
4,SMESG000000025.1,4.740000e-11,73.6,"KOG3720@1|root,KOG3720@2759|Eukaryota,37X8R@33...",3041|Chlorophyta,Belongs to the histidine acid phosphatase family,-,-,3.1.3.2,ko:K14395,His_Phos_2,Smed,None,None,KOG3720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15503,SMESG000081975.1,1.880000e-91,288.0,"COG0491@1|root,KOG0813@2759|Eukaryota,38RE4@33...",33208|Metazoa,hydroxyacylglutathione hydrolase activity,HAGH,"GO:0000003,GO:0001666,GO:0003674,GO:0003824,GO...",3.1.2.6,ko:K01069,"HAGH_C,Lactamase_B",Smed,3BIVM,3CRVK,KOG0813
15504,SMESG000081983.1,1.180000e-152,440.0,"KOG0082@1|root,KOG0082@2759|Eukaryota,38C4P@33...",33208|Metazoa,It is involved in the biological process descr...,-,"GO:0000132,GO:0000166,GO:0000226,GO:0000278,GO...",-,ko:K04630,G-alpha,Smed,3BASG,3CUQY,KOG0082
15505,SMESG000081985.1,1.490000e-39,146.0,"2D16C@1|root,2SGW5@2759|Eukaryota,3ANBW@33154|...",33208|Metazoa,hAT family C-terminal dimerisation region,-,-,-,-,Dimer_Tnp_hAT,Smed,3C1V9,3DBZC,2SGW5
15506,SMESG000081986.1,2.720000e-04,50.4,"COG2801@1|root,KOG0017@2759|Eukaryota,382V8@33...",33090|Viridiplantae,Zinc knuckle,-,-,-,-,zf-CCHC,Smed,None,None,KOG0017


In [7]:
sman = pd.read_table( 'Refs/SM_V7/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
sman = sman.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
sman['Species'] = 'Sman'
sman['Metazoa'] = sman.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
sman['Bilateria'] = sman.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
sman['Eukaryota'] = sman.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
sman

/tmp/ipykernel_32011/1741277779.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  sman = pd.read_table( '../../Refs/SM_V7/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,Smp_000020.1,0.000000e+00,2167.0,"KOG3698@1|root,KOG3698@2759|Eukaryota,38D91@33...",33208|Metazoa,[protein]-3-O-(N-acetyl-D-glucosaminyl)-L-seri...,MGEA5,"GO:0002119,GO:0002164,GO:0002791,GO:0002793,GO...","2.3.1.48,3.2.1.169",ko:K15719,NAGidase,Sman,3BC69,3CUMB,KOG3698
1,Smp_000030.1,1.530000e-165,490.0,"COG5110@1|root,KOG2005@2759|Eukaryota,38ER5@33...",33208|Metazoa,regulation of protein catabolic process,PSMD2,"GO:0000003,GO:0000502,GO:0001775,GO:0002119,GO...",-,ko:K03028,PC_rep,Sman,3BEI6,3CRE4,KOG2005
2,Smp_000030.2,0.000000e+00,1916.0,"COG5110@1|root,KOG2005@2759|Eukaryota,38ER5@33...",33208|Metazoa,regulation of protein catabolic process,PSMD2,"GO:0000003,GO:0000502,GO:0001775,GO:0002119,GO...",-,ko:K03028,PC_rep,Sman,3BEI6,3CRE4,KOG2005
3,Smp_000030.3,0.000000e+00,1398.0,"COG5110@1|root,KOG2005@2759|Eukaryota,38ER5@33...",33208|Metazoa,regulation of protein catabolic process,PSMD2,"GO:0000003,GO:0000502,GO:0001775,GO:0002119,GO...",-,ko:K03028,PC_rep,Sman,3BEI6,3CRE4,KOG2005
4,Smp_000040.1,2.850000e-123,374.0,"COG0457@1|root,KOG1840@2759|Eukaryota,38CFM@33...",33208|Metazoa,microtubule motor activity,KLC4,"GO:0000003,GO:0000226,GO:0000280,GO:0000902,GO...",-,ko:K10407,"DUF2315,TPR_10,TPR_12,TPR_7,TPR_8",Sman,3BC82,3CWNT,KOG1840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11693,Smp_900050.1,1.500000e-315,865.0,"COG1007@1|root,KOG4668@2759|Eukaryota,38F5W@33...",33208|Metazoa,NADH dehydrogenase (quinone) activity,ND5,"GO:0000302,GO:0001666,GO:0003674,GO:0003824,GO...",1.6.5.3,ko:K03883,"NADH5_C,Proton_antipo_M,Proton_antipo_N",Sman,3BEVW,3CTS4,KOG4668
11694,Smp_900060.1,3.510000e-12,69.3,"COG1845@1|root,KOG4664@2759|Eukaryota",2759|Eukaryota,aerobic electron transport chain,COX3,"GO:0003674,GO:0003824,GO:0004129,GO:0005215,GO...",-,ko:K02262,COX3,Sman,None,None,KOG4664
11695,Smp_900070.1,1.230000e-220,612.0,"COG1290@1|root,KOG4663@2759|Eukaryota,38FBZ@33...",33208|Metazoa,Component of the ubiquinol-cytochrome c reduct...,CYTB,"GO:0000003,GO:0001666,GO:0003674,GO:0003824,GO...",-,ko:K00412,"Cytochrom_B_C,Cytochrome_B",Sman,3BCGI,3CZ4H,KOG4663
11696,Smp_900090.1,1.460000e-257,711.0,"COG1008@1|root,KOG4845@2759|Eukaryota",2759|Eukaryota,NADH dehydrogenase (quinone) activity,psaC,"GO:0003674,GO:0003824,GO:0003954,GO:0005575,GO...",1.6.5.3,"ko:K02691,ko:K03881","Fer4,Proton_antipo_M",Sman,None,None,KOG4845


In [8]:
mcli = pd.read_table( 'Refs/Maccli_GV23d_v2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
mcli = mcli.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
mcli['Species'] = 'Mcli'
mcli['Metazoa'] = mcli.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
mcli['Bilateria'] = mcli.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
mcli['Eukaryota'] = mcli.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
mcli

/tmp/ipykernel_32011/3043471437.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  mcli = pd.read_table( '../../Refs/Maccli_GV23d_v2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,Mcli000051.g.t1,1.530000e-78,238.0,"KOG4028@1|root,KOG4028@2759|Eukaryota,39TFZ@33...",33208|Metazoa,B9 domain-containing protein 2,B9D2,"GO:0000003,GO:0000226,GO:0001578,GO:0001736,GO...",-,"ko:K16745,ko:K19382","B9-C2,DUF1619",Mcli,3BEJ3,3CYGJ,KOG4028
1,Mcli000095.g.t1,6.610000e-69,268.0,"COG1643@1|root,KOG1812@1|root,KOG0922@2759|Euk...",35493|Streptophyta,Helicase associated domain (HA2) Add an annot...,-,-,3.6.4.13,ko:K12818,"DEAD,HA2,Helicase_C,IBR,OB_NTP_bind",Mcli,None,None,KOG0922
2,Mcli000010.g.t1,3.950000e-62,216.0,"COG2801@1|root,KOG0017@2759|Eukaryota,38F42@33...",33208|Metazoa,retrotransposable element Tf2 155 kDa protein ...,-,-,-,-,"RVT_1,rve,zf-H2C2",Mcli,3BI8I,3D5CW,KOG0017
3,Mcli000016.g.t1,3.260000e-05,51.6,"2BP4M@1|root,2S1R2@2759|Eukaryota,3A5HU@33154|...",33208|Metazoa,-,-,-,-,-,-,Mcli,3BRC0,3D8S6,2S1R2
4,Mcli000090.g.t1,1.640000e-07,60.1,"COG2801@1|root,KOG0017@2759|Eukaryota,38F42@33...",33208|Metazoa,retrotransposable element Tf2 155 kDa protein ...,-,-,-,-,"RVT_1,rve",Mcli,3BI8I,3D5CW,KOG0017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23633,Mcli038771.g.t1,3.260000e-04,49.7,"KOG4297@1|root,KOG4297@2759|Eukaryota",2759|Eukaryota,carbohydrate binding,-,-,-,-,Lectin_C,Mcli,None,None,KOG4297
23634,Mcli038769.g.t1,1.500000e-07,62.8,"2BFTC@1|root,2S17S@2759|Eukaryota,39MPG@33154|...",33208|Metazoa,Glycosyltransferase sugar-binding region conta...,-,-,-,-,Gly_transf_sug,Mcli,3BS19,3D87K,2S17S
23635,Mcli038772.g.t1,1.200000e-16,92.8,"KOG3547@1|root,KOG3547@2759|Eukaryota,38DHP@33...",33208|Metazoa,chloride channel activity,BEST3,"GO:0003674,GO:0005215,GO:0005216,GO:0005253,GO...",-,"ko:K13878,ko:K13879,ko:K13880,ko:K13881,ko:K22204",Bestrophin,Mcli,3B96Y,3CS5T,KOG3547
23636,Mcli038766.g.t1,2.520000e-07,53.1,"2CUJS@1|root,2S4E7@2759|Eukaryota,3A7GX@33154|...",33208|Metazoa,vitamin-K-epoxide reductase (warfarin-sensitiv...,VKORC1L1,"GO:0003674,GO:0003824,GO:0005575,GO:0005622,GO...",1.17.4.4,ko:K05357,VKOR,Mcli,3BT0U,3D254,2S4E7


In [9]:
mhys = pd.read_table( 'Refs/Machtx_SR1_v2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
mhys = mhys.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
mhys['Species'] = 'Mhys'
mhys['Metazoa'] = mhys.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
mhys['Bilateria'] = mhys.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
mhys['Eukaryota'] = mhys.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
mhys

/tmp/ipykernel_32011/2845957693.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  mhys = pd.read_table( '../../Refs/Machtx_SR1_v2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,Mhtx000148.g.t1,1.670000e-30,115.0,"COG1358@1|root,KOG3167@2759|Eukaryota,3A0G7@33...",33208|Metazoa,Ribosomal protein L7Ae/L30e/S12e/Gadd45 family,NHP2,"GO:0000154,GO:0000469,GO:0000470,GO:0000723,GO...",2.7.11.22,"ko:K02449,ko:K11129",Ribosomal_L7Ae,Mhys,3BPVH,3D6NR,KOG3167
1,Mhtx000062.g.t1,8.310000e-59,202.0,"COG1358@1|root,KOG1721@1|root,KOG1721@2759|Euk...",33208|Metazoa,Zinc finger protein,-,"GO:0000785,GO:0000792,GO:0002009,GO:0002165,GO...",-,ko:K09228,"zf-C2H2,zf-C2H2_6,zf-met",Mhys,3BA9A,3CT35,KOG1721
2,Mhtx000141.g.t1,4.770000e-26,107.0,"KOG0041@1|root,KOG0041@2759|Eukaryota,39GAQ@33...",33208|Metazoa,"EF-hand, calcium binding motif",EFHD1,"GO:0003674,GO:0005488,GO:0005509,GO:0005575,GO...",-,-,"EF-hand_1,EF-hand_5,EF-hand_6,EF-hand_7",Mhys,3BHRZ,3CXRD,KOG0041
3,Mhtx000005.g.t1,9.930000e-17,76.3,"2BXE4@1|root,2SF2X@2759|Eukaryota,3A8T7@33154|...",33208|Metazoa,Cystatin-like domain,CSTA,-,-,ko:K13907,Cystatin,Mhys,3BUN6,3DC6X,2SF2X
4,Mhtx000136.g.t2,1.740000e-235,748.0,"COG0666@1|root,KOG3609@1|root,KOG3609@2759|Euk...",33208|Metazoa,calcium channel activity,-,"GO:0001964,GO:0003008,GO:0003674,GO:0005215,GO...",-,-,"Ank,Ank_2,Ank_3,Ank_4,Ion_trans",Mhys,3B9FZ,3CWTC,KOG3609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21599,Mhtx034263.g.t1,1.750000e-87,275.0,"KOG0761@1|root,KOG0761@2759|Eukaryota,38GBT@33...",33208|Metazoa,mitochondrial transport,SLC25A40,"GO:0000041,GO:0003674,GO:0005215,GO:0005575,GO...",-,ko:K15119,Mito_carr,Mhys,3BE8D,3CYZP,KOG0761
21600,Mhtx034409.g.t1,9.520000e-24,114.0,"KOG4225@1|root,KOG4225@2759|Eukaryota,38EJB@33...",33208|Metazoa,positive regulation of membrane protein ectodo...,SH3D19,"GO:0003674,GO:0005488,GO:0005515,GO:0005575,GO...",-,-,"SH3_1,SH3_2,SH3_9",Mhys,3BCRX,3CRWW,KOG4225
21601,Mhtx033695.g.t1,4.210000e-310,926.0,"COG0553@1|root,KOG0383@2759|Eukaryota,38BGB@33...",33208|Metazoa,It is involved in the biological process descr...,CHD5,"GO:0000003,GO:0000118,GO:0000122,GO:0000226,GO...",3.6.4.12,"ko:K11642,ko:K11643,ko:K14435","CHDCT2,CHDNT,Chromo,DUF1086,DUF1087,Helicase_C...",Mhys,3B940,3CVFP,KOG0383
21602,Mhtx034464.g.t1,1.900000e-93,346.0,"2CMP4@1|root,2QR56@2759|Eukaryota,39VQX@33154|...",33208|Metazoa,Pao retrotransposon peptidase,-,-,-,-,"DUF1758,DUF1759,Peptidase_A17,RVT_1,rve",Mhys,3BR69,None,2QR56


In [10]:
spol = pd.read_table( 'Refs/dd_Spol_v4/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
spol = spol.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
spol['Species'] = 'Spol'
spol['Metazoa'] = spol.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
spol['Bilateria'] = spol.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
spol['Eukaryota'] = spol.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
spol

/tmp/ipykernel_32011/3063053755.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  spol = pd.read_table( '../../Refs/dd_Spol_v4/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,dd_Spol_v4_10002_1_1.p1,4.290000e-97,311.0,"KOG3656@1|root,KOG3656@2759|Eukaryota,38EWX@33...",33208|Metazoa,alpha1-adrenergic receptor activity,-,"GO:0003674,GO:0004888,GO:0004930,GO:0004952,GO...",-,-,7tm_1,Spol,3BB8F,3CVUK,KOG3656
1,dd_Spol_v4_10006_1_1.p1,0.000000e+00,1053.0,"COG0326@1|root,KOG0019@2759|Eukaryota,38B7K@33...",33208|Metazoa,Heat shock protein,-,"GO:0001101,GO:0002119,GO:0002164,GO:0003674,GO...",-,ko:K04079,"HATPase_c,HSP90",Spol,3BFAG,3CVM1,KOG0019
2,dd_Spol_v4_10007_1_1.p1,5.280000e-152,475.0,"COG0666@1|root,KOG0505@2759|Eukaryota",2759|Eukaryota,positive regulation of myosin-light-chain-phos...,ANKS6,-,-,"ko:K06270,ko:K17458,ko:K21415","Ank,Ank_2,Ank_3,Ank_4,PRKG1_interact,RabGAP-TB...",Spol,None,None,KOG0505
3,dd_Spol_v4_10013_1_1.p1,2.560000e-173,498.0,"KOG2740@1|root,KOG2740@2759|Eukaryota,38ENS@33...",33208|Metazoa,synaptic vesicle cytoskeletal transport,AP3M2,"GO:0000323,GO:0003674,GO:0005215,GO:0005488,GO...",-,ko:K12398,"Adap_comp_sub,Clat_adaptor_s",Spol,3BADW,3CT9H,KOG2740
4,dd_Spol_v4_10014_1_1.p1,8.830000e-15,85.9,"KOG3683@1|root,KOG3683@2759|Eukaryota,39SUU@33...",33208|Metazoa,"RMI1, RecQ mediated genome instability 1, homo...",RMI1,"GO:0002021,GO:0002023,GO:0005575,GO:0005622,GO...",2.3.2.27,"ko:K10990,ko:K15700","RMI1_C,RMI1_N",Spol,3BDYU,3CUPH,KOG3683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28978,dd_Spol_v4_9996_3_2.p1,8.270000e-69,223.0,"KOG0082@1|root,KOG0099@2759|Eukaryota,38C3N@33...",33208|Metazoa,It is involved in the biological process descr...,GNAL,"GO:0000003,GO:0000139,GO:0000166,GO:0001101,GO...",-,"ko:K04632,ko:K04633",G-alpha,Spol,3B9MU,3CWC5,KOG0099
28979,dd_Spol_v4_9996_3_3.p1,8.270000e-69,223.0,"KOG0082@1|root,KOG0099@2759|Eukaryota,38C3N@33...",33208|Metazoa,It is involved in the biological process descr...,GNAL,"GO:0000003,GO:0000139,GO:0000166,GO:0001101,GO...",-,"ko:K04632,ko:K04633",G-alpha,Spol,3B9MU,3CWC5,KOG0099
28980,dd_Spol_v4_9996_3_3.p2,4.170000e-115,340.0,"KOG0082@1|root,KOG0099@2759|Eukaryota,38C3N@33...",33208|Metazoa,It is involved in the biological process descr...,GNAL,"GO:0000003,GO:0000139,GO:0000166,GO:0001101,GO...",-,"ko:K04632,ko:K04633",G-alpha,Spol,3B9MU,3CWC5,KOG0099
28981,dd_Spol_v4_99_1_1.p1,8.220000e-14,83.6,"KOG1032@1|root,KOG1032@2759|Eukaryota,38D80@33...",4751|Fungi,"domain in glucosyltransferases, myotubularins ...",-,"GO:0003674,GO:0005488,GO:0005496,GO:0005575,GO...",-,-,"DUF4782,GRAM",Spol,None,None,KOG1032


In [11]:
sjap = pd.read_table( 'Refs/HuSjv2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
sjap = sjap.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
sjap['Species'] = 'Sjap'
sjap['Metazoa'] = sjap.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
sjap['Bilateria'] = sjap.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
sjap['Eukaryota'] = sjap.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
sjap

/tmp/ipykernel_32011/3156094850.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  sjap = pd.read_table( '../../Refs/HuSjv2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,EWB00_000003.1,5.390000e-153,454.0,"KOG2130@1|root,KOG2130@2759|Eukaryota,38DTH@33...",33208|Metazoa,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,JMJD6,"GO:0001525,GO:0001568,GO:0001654,GO:0001655,GO...",-,ko:K11323,JmjC,Sjap,3BBGB,3CYZ7,KOG2130
1,EWB00_000003.2,8.660000e-203,585.0,"KOG2130@1|root,KOG2130@2759|Eukaryota,38DTH@33...",33208|Metazoa,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,JMJD6,"GO:0001525,GO:0001568,GO:0001654,GO:0001655,GO...",-,ko:K11323,JmjC,Sjap,3BBGB,3CYZ7,KOG2130
2,EWB00_000003.3,5.790000e-168,493.0,"KOG2130@1|root,KOG2130@2759|Eukaryota,38DTH@33...",33208|Metazoa,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,JMJD6,"GO:0001525,GO:0001568,GO:0001654,GO:0001655,GO...",-,ko:K11323,JmjC,Sjap,3BBGB,3CYZ7,KOG2130
3,EWB00_000004,9.260000e-16,90.5,"KOG2130@1|root,KOG2130@2759|Eukaryota,38DTH@33...",33208|Metazoa,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,JMJD6,"GO:0001525,GO:0001568,GO:0001654,GO:0001655,GO...",-,ko:K11323,JmjC,Sjap,3BBGB,3CYZ7,KOG2130
4,EWB00_000006,3.560000e-65,224.0,"KOG2130@1|root,KOG2130@2759|Eukaryota,38DTH@33...",33208|Metazoa,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,JMJD6,"GO:0001525,GO:0001568,GO:0001654,GO:0001655,GO...",-,ko:K11323,JmjC,Sjap,3BBGB,3CYZ7,KOG2130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13677,EWB00_011420,2.420000e-143,428.0,"COG0814@1|root,KOG1305@2759|Eukaryota,38RV5@33...",33208|Metazoa,"Solute carrier family 38, member 9",SLC38A9,"GO:0000323,GO:0001101,GO:0003333,GO:0003674,GO...",-,ko:K14995,Aa_trans,Sjap,3BDWT,3CX0V,KOG1305
13678,EWB00_011422.1,3.050000e-08,58.5,"2CXHJ@1|root,2RXJR@2759|Eukaryota,3A3G0@33154|...",33208|Metazoa,Heme binding protein 2,HEBP2,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",-,-,SOUL,Sjap,3BRJH,3D1CK,2RXJR
13679,EWB00_011422.2,3.050000e-08,58.5,"2CXHJ@1|root,2RXJR@2759|Eukaryota,3A3G0@33154|...",33208|Metazoa,Heme binding protein 2,HEBP2,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",-,-,SOUL,Sjap,3BRJH,3D1CK,2RXJR
13680,EWB00_011426,2.660000e-172,492.0,"COG0596@1|root,KOG2624@2759|Eukaryota,38E9D@33...",33208|Metazoa,"hydrolase activity, acting on ester bonds",-,-,3.1.1.13,ko:K01052,"Abhydro_lipase,Abhydrolase_1,Hydrolase_4",Sjap,3BA6C,3CTTI,KOG2624


In [12]:
plei = pd.read_table( 'Refs/Plei_AlvarezCampos2023/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
plei = plei.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
plei['Species'] = 'Plei'
plei['Metazoa'] = plei.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
plei['Bilateria'] = plei.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
plei['Eukaryota'] = plei.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
plei

/tmp/ipykernel_32011/177220056.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  plei = pd.read_table( '../../Refs/Plei_AlvarezCampos2023/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,PrileiEVm000001t1.p1,0.000000e+00,3859.0,"COG5184@1|root,KOG1426@2759|Eukaryota,38ETC@33...",33208|Metazoa,SUMO binding,HERC2,"GO:0000003,GO:0000726,GO:0003674,GO:0003824,GO...",2.3.2.26,ko:K10595,"ANAPC10,Cul7,Cyt-b5,HECT,MIB_HERC2,RCC1,ZZ",Plei,3BIJT,3CS5U,KOG1426
1,PrileiEVm000002t1.p1,8.820000e-129,468.0,"KOG4338@1|root,KOG4338@2759|Eukaryota,38D8A@33...",33208|Metazoa,Apolipoprotein,-,"GO:0003674,GO:0005102,GO:0005215,GO:0005319,GO...",-,-,"C8,DUF1081,DUF1943,VWD,Vitellogenin_N",Plei,3BCAR,3CYWB,KOG4338
2,PrileiEVm000003t1.p1,0.000000e+00,2133.0,"COG5032@1|root,KOG0891@2759|Eukaryota,38GFS@33...",33208|Metazoa,Serine threonine-protein kinase SMG1,SMG1,"GO:0000003,GO:0000184,GO:0000956,GO:0003006,GO...",2.7.11.1,ko:K08873,"DUF5303,FATC,PI3_PI4_kinase,SMG1",Plei,3BB56,3CV6H,KOG0891
3,PrileiEVm000004t1.p1,3.510000e-06,60.1,"KOG1217@1|root,KOG1217@2759|Eukaryota,38BUP@33...",33208|Metazoa,negative regulation of growth rate,-,-,-,ko:K02599,"Ank_2,Ank_4,EGF,EGF_CA,Notch,hEGF",Plei,3BAFT,3CRZB,KOG1217
4,PrileiEVm000005t1.p1,0.000000e+00,2161.0,"KOG1836@1|root,KOG1836@2759|Eukaryota,38DXH@33...",33208|Metazoa,laminin subunit,epi-1,"GO:0000003,GO:0000902,GO:0000904,GO:0001764,GO...",-,ko:K06240,"Laminin_B,Laminin_EGF,Laminin_G_2,Laminin_I,La...",Plei,3BCFJ,3CRYA,KOG1836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18563,PrileiEVm029318t1.p1,2.840000e-22,90.9,"2BXE4@1|root,2SF2X@2759|Eukaryota,3A7JA@33154|...",33208|Metazoa,Leukocyte cysteine proteinase inhibitor 1-like,CSTB,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",-,ko:K13907,Cystatin,Plei,3BSI3,3DA4H,2SF2X
18564,PrileiEVm029992t1.p1,2.610000e-53,181.0,"COG0499@1|root,KOG1370@2759|Eukaryota,38CN8@33...",33208|Metazoa,"S-adenosyl-L-homocysteine hydrolase, NAD bindi...",AHCYL1,"GO:0000096,GO:0002028,GO:0003674,GO:0005488,GO...",3.3.1.1,ko:K01251,"AdoHcyase,AdoHcyase_NAD",Plei,3BE96,3CSQJ,KOG1370
18565,PrileiEVm029999t1.p1,1.020000e-53,169.0,"COG1997@1|root,KOG0402@2759|Eukaryota,3A5SK@33...",33208|Metazoa,structural constituent of ribosome,RPL37A,"GO:0000184,GO:0000956,GO:0002181,GO:0003674,GO...",-,ko:K02921,Ribosomal_L37ae,Plei,3BRQ5,3D8GG,KOG0402
18566,PrileiEVm030012t1.p1,4.000000e-35,122.0,"KOG4604@1|root,KOG4604@2759|Eukaryota,3A8K4@33...",33208|Metazoa,MICOS complex,MINOS1,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",-,ko:K17784,DUF543,Plei,3BSID,3D9CK,KOG4604


In [13]:
pdum = pd.read_table( 'Refs/Pdum_v2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
pdum = pdum.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
pdum['Species'] = 'Pdum'
pdum['Metazoa'] = pdum.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
pdum['Bilateria'] = pdum.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
pdum['Eukaryota'] = pdum.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
pdum

/tmp/ipykernel_32011/616097271.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pdum = pd.read_table( '../../Refs/Pdum_v2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,comp0_c0_seq1.p1,6.550000e-05,47.8,"COG2453@1|root,KOG1716@2759|Eukaryota",2759|Eukaryota,protein tyrosine/serine/threonine phosphatase ...,DUSP19,"GO:0000165,GO:0000188,GO:0001932,GO:0001933,GO...","3.1.3.16,3.1.3.48","ko:K01104,ko:K14165","DSPc,PseudoU_synth_1",Pdum,None,None,KOG1716
1,comp1000916_c0_seq1.p1,2.670000e-11,66.6,"KOG1075@1|root,KOG1075@2759|Eukaryota,39N4Q@33...",33208|Metazoa,Reverse transcriptase (RNA-dependent DNA polym...,-,-,-,-,RVT_1,Pdum,3B9WN,3D1S8,KOG1075
2,comp10013_c0_seq1.p1,9.060000e-10,62.4,"2BWHM@1|root,2QU1C@2759|Eukaryota,39X3N@33154|...",33208|Metazoa,Innexin,-,"GO:0005575,GO:0005911,GO:0005921,GO:0030054",-,ko:K22037,Innexin,Pdum,3BJDA,3D45Z,2QU1C
3,comp1001628_c0_seq1.p2,3.630000e-87,263.0,"COG1024@1|root,COG1024@2|Bacteria,2GMET@201174...",201174|Actinobacteria,Enoyl-CoA hydratase/isomerase,echA9,-,4.2.1.17,ko:K01692,ECH_2,Pdum,None,None,None
4,comp10037_c0_seq1.p1,1.320000e-95,303.0,"KOG3644@1|root,KOG3642@2759|Eukaryota,39MCE@33...",2759|Eukaryota,Neurotransmitter-gated ion-channel ligand bind...,-,-,-,-,"Neur_chan_LBD,Neur_chan_memb",Pdum,3CNZG,3E539,KOG3642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42685,comp997096_c0_seq1.p1,7.720000e-05,48.5,"KOG2122@1|root,KOG2122@2759|Eukaryota,39IR6@33...",33208|Metazoa,beta-catenin binding,Apc,"GO:0000003,GO:0001704,GO:0001706,GO:0001708,GO...",-,ko:K02085,"APC_r,Arm,SAMP",Pdum,3BB0V,3CU67,KOG2122
42686,comp99832_c0_seq1.p1,1.070000e-82,247.0,"COG1309@1|root,COG1309@2|Bacteria,2HAGC@201174...",201174|Actinobacteria,"Tetracyclin repressor, C-terminal all-alpha do...",tetR,-,-,-,"TetR_C,TetR_N",Pdum,None,None,None
42687,comp9985_c1_seq1.p1,1.050000e-11,67.4,"KOG1565@1|root,KOG1565@2759|Eukaryota,39SEW@33...",33208|Metazoa,Belongs to the peptidase M10A family,MMP9,"GO:0000003,GO:0000302,GO:0001101,GO:0001501,GO...",3.4.24.35,ko:K01403,"Hemopexin,PG_binding_1,PT,Peptidase_M10,fn2",Pdum,3B9DD,3CVA2,KOG1565
42688,comp9989_c0_seq1.p1,4.580000e-88,259.0,"COG3731@1|root,COG3731@2|Bacteria,2GT3D@201174...",201174|Actinobacteria,PTS system glucitol/sorbitol-specific IIA comp...,-,-,2.7.1.198,ko:K02781,PTSIIA_gutA,Pdum,None,None,None


In [14]:
clon = pd.read_table( 'Refs/Clong/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
clon = clon.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
clon['Species'] = 'Clon'
clon['Metazoa'] = clon.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
clon['Bilateria'] = clon.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
clon['Eukaryota'] = clon.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
clon

/tmp/ipykernel_32011/262761276.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  clon = pd.read_table( '../../Refs/Clong/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,cluster_534_pilon,2.870000e-08,67.8,"COG5069@1|root,KOG0517@2759|Eukaryota,3AH5E@33...",33208|Metazoa,Spectrin repeat,SPTBN5,"GO:0000003,GO:0000165,GO:0000902,GO:0000904,GO...",-,ko:K06115,"CH,PH,PH_9,SH3_2,SH3_9,Spectrin",Clon,3BCCD,3CXWC,KOG0517
1,cluster_9753_pilon,1.630000e-47,177.0,"KOG2895@1|root,KOG2895@2759|Eukaryota,37PE0@33...",35493|Streptophyta,membrane protein C776.05-like,-,-,-,-,DUF2838,Clon,None,None,KOG2895
2,cluster_6244_pilon,2.090000e-26,119.0,"COG1519@1|root,2QSA5@2759|Eukaryota,37S0U@3309...",35493|Streptophyta,3-deoxy-D-manno-octulosonic acid,-,-,"2.4.99.12,2.4.99.13,2.4.99.14,2.4.99.15",ko:K02527,Glycos_transf_N,Clon,None,None,2QSA5
3,cluster_5977_pilon,7.400000e-12,78.2,"COG4642@1|root,KOG0231@2759|Eukaryota,38BBY@33...",33208|Metazoa,Radial spoke head 10 homolog,RSPH10B,-,-,-,MORN,Clon,3B99A,3D1H8,KOG0231
4,cluster_24816_pilon,6.400000e-15,76.3,"2CH2M@1|root,2S05S@2759|Eukaryota,3A17Y@33154|...",33208|Metazoa,GM2 ganglioside activator,GM2A,"GO:0000323,GO:0001573,GO:0001775,GO:0002252,GO...",-,ko:K12383,E1_DerP2_DerF2,Clon,3BQ49,3D6QA,2S05S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13747,cluster_13676_pilon,2.360000e-240,677.0,"COG0130@1|root,KOG2529@2759|Eukaryota,37JSV@33...",3041|Chlorophyta,DKCLD (NUC011) domain,CBF5,-,-,ko:K11131,"DKCLD,PUA,TruB_C_2,TruB_N",Clon,None,None,KOG2529
13748,cluster_3598_pilon,3.820000e-16,84.7,"KOG1052@1|root,KOG1052@2759|Eukaryota,39UC9@33...",33208|Metazoa,Belongs to the glutamate-gated ion channel (TC...,-,"GO:0005575,GO:0005623,GO:0005886,GO:0016020,GO...",-,-,"Lig_chan,Lig_chan-Glu_bd",Clon,3BIK1,3D0AN,KOG1052
13749,cluster_10094_pilon,7.970000e-157,463.0,"KOG0255@1|root,KOG0255@2759|Eukaryota,38BDZ@33...",33208|Metazoa,neurotransmitter transport,-,"GO:0003674,GO:0005215,GO:0005575,GO:0005623,GO...",-,ko:K06258,"MFS_1,Sugar_tr",Clon,3B9V0,3CVDU,KOG0255
13750,cluster_17917_pilon,5.350000e-07,59.7,"KOG4818@1|root,KOG4818@2759|Eukaryota,38CD8@33...",33208|Metazoa,positive regulation of natural killer cell deg...,LAMP1,"GO:0000003,GO:0000323,GO:0001775,GO:0001910,GO...",-,ko:K06528,Lamp,Clon,3BA74,3D266,KOG4818


In [15]:
hmia = pd.read_table( 'Refs/Clong/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
hmia = hmia.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
hmia['Species'] = 'Hmia'
hmia['Metazoa'] = hmia.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
hmia['Bilateria'] = hmia.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
hmia['Eukaryota'] = hmia.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
hmia

/tmp/ipykernel_32011/2215851116.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  hmia = pd.read_table( '../../Refs/HmiaM1/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,HMIM017694-PA,3.630000e-12,73.2,"28I8Q@1|root,2QQJ1@2759|Eukaryota,38DI2@33154|...",33208|Metazoa,Methyltransferase domain,HNMT,"GO:0001505,GO:0001692,GO:0001695,GO:0002347,GO...",2.1.1.8,ko:K00546,Methyltransf_23,Hmia,3BCPH,3CXTQ,2QQJ1
1,HMIM011485-PA,2.450000e-84,285.0,"KOG4217@1|root,KOG4217@2759|Eukaryota,39DCT@33...",33208|Metazoa,"Nuclear receptor subfamily 4, group A, member 1",NR4A1,"GO:0000981,GO:0000982,GO:0001077,GO:0001228,GO...",-,ko:K04465,"Hormone_recep,zf-C4",Hmia,3BGP0,3D0VB,KOG4217
2,HMIM017964-PA,7.460000e-34,130.0,"COG5241@1|root,KOG2841@2759|Eukaryota,38Y6Z@33...",4751|Fungi,DNA repair endonuclease Swi10,RAD10,"GO:0000003,GO:0000014,GO:0000109,GO:0000110,GO...",-,ko:K10849,"HHH_5,Rad10",Hmia,None,None,KOG2841
3,HMIM016607-PA,4.700000e-79,267.0,"KOG2433@1|root,KOG2433@2759|Eukaryota,38GNJ@33...",33208|Metazoa,Ufm1-specific peptidase 2,UFSP2,"GO:0003674,GO:0003824,GO:0005575,GO:0005622,GO...",-,ko:K01376,Peptidase_C78,Hmia,3BG1X,3CS7R,KOG2433
4,HMIM010080-PA,4.200000e-174,515.0,"COG1283@1|root,2QQ3I@2759|Eukaryota,38DHE@3315...",33208|Metazoa,sodium-dependent phosphate transmembrane trans...,SLC34A2,"GO:0001503,GO:0001655,GO:0001701,GO:0001822,GO...",-,ko:K14683,Na_Pi_cotrans,Hmia,3BA55,3CRFS,2QQ3I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15682,HMIM014277-PA,4.440000e-14,85.5,"KOG4356@1|root,KOG4356@2759|Eukaryota,38HR9@33...",33208|Metazoa,cilium-dependent cell motility,DNAAF2,"GO:0000226,GO:0001101,GO:0001539,GO:0001578,GO...",-,ko:K19751,PIH1,Hmia,3BBM0,3CUPJ,KOG4356
15683,HMIM008815-PA,7.230000e-155,469.0,"COG2940@1|root,KOG1079@2759|Eukaryota,38DMC@33...",33208|Metazoa,polycomb repressive complex 2 subunit,EZH1,"GO:0000003,GO:0000122,GO:0000228,GO:0000278,GO...",2.1.1.43,"ko:K11430,ko:K17451","EZH2_WD-Binding,SET",Hmia,3BB21,3CR6U,KOG1079
15684,HMIM017871-PA,2.170000e-17,85.1,"COG5126@1|root,KOG0028@2759|Eukaryota,38C1B@33...",33208|Metazoa,EF-hand domain,CETN2,"GO:0003674,GO:0005488,GO:0005509,GO:0005575,GO...",-,"ko:K10840,ko:K16465","EF-hand_1,EF-hand_5,EF-hand_6,EF-hand_7,EF-hand_8",Hmia,3BB2J,3D6WC,KOG0028
15685,HMIM021086-PA,1.820000e-11,68.2,"28IIP@1|root,2QQVP@2759|Eukaryota,39TA7@33154|...",33208|Metazoa,Domain of unknown function (DUF4371),-,-,-,-,"DUF4371,Dimer_Tnp_hAT",Hmia,3CNWS,3D3DK,2QQVP


In [16]:
nvec = pd.read_table( 'Refs/wein_nvec200_tcsv2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
nvec = nvec.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
nvec['Species'] = 'Nvec'
nvec['Metazoa'] = nvec.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
nvec['Bilateria'] = nvec.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
nvec['Eukaryota'] = nvec.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
nvec

/tmp/ipykernel_32011/733553463.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  nvec = pd.read_table( '../../Refs/wein_nvec200_tcsv2/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,NV2t000003001.1,5.020000e-28,131.0,"2BY2G@1|root,2QPW4@2759|Eukaryota,38FBN@33154|...",33208|Metazoa,regulation of superoxide dismutase activity,SZT2,"GO:0000323,GO:0000902,GO:0000904,GO:0005575,GO...",-,-,-,Nvec,3BBY1,3CS7S,2QPW4
1,NV2t000005001.1,7.270000e-231,694.0,"2BY2G@1|root,2QPW4@2759|Eukaryota,38FBN@33154|...",33208|Metazoa,corpus callosum morphogenesis,-,-,-,-,-,Nvec,3BBY1,None,2QPW4
2,NV2t000006001.1,3.140000e-135,410.0,"2BY2G@1|root,2QPW4@2759|Eukaryota,38FBN@33154|...",33208|Metazoa,corpus callosum morphogenesis,-,-,-,-,-,Nvec,3BBY1,None,2QPW4
3,NV2t000007001.1,8.760000e-294,801.0,"28KM0@1|root,2QT2E@2759|Eukaryota,39073@33154|...",33208|Metazoa,Transmembrane protein 5,TMEM5,"GO:0003674,GO:0003824,GO:0005575,GO:0005622,GO...",-,ko:K21052,Exostosin,Nvec,3BE01,None,2QT2E
4,NV2t000008001.1,2.440000e-60,197.0,"KOG3583@1|root,KOG3583@2759|Eukaryota,39S7D@33...",33208|Metazoa,nucleic acid-templated transcription,MED8,"GO:0003674,GO:0003712,GO:0005575,GO:0005622,GO...",-,ko:K15129,Med8,Nvec,3BHDQ,None,KOG3583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25690,NV2t025984002.1,1.510000e-71,239.0,"KOG0977@1|root,KOG0977@2759|Eukaryota",2759|Eukaryota,intermediate filament organization,LMNTD2,"GO:0005575,GO:0005622,GO:0005623,GO:0005634,GO...",-,ko:K07611,LTD,Nvec,None,None,KOG0977
25691,NV2t025984003.1,5.260000e-74,248.0,"KOG0977@1|root,KOG0977@2759|Eukaryota",2759|Eukaryota,intermediate filament organization,LMNTD2,"GO:0005575,GO:0005622,GO:0005623,GO:0005634,GO...",-,ko:K07611,LTD,Nvec,None,None,KOG0977
25692,NV2t025985001.1,4.740000e-49,157.0,"KOG0977@1|root,KOG0977@2759|Eukaryota,39VDT@33...",33208|Metazoa,structural molecule activity,LMNTD1,"GO:0005575,GO:0005622,GO:0005623,GO:0005634,GO...",-,-,LTD,Nvec,3BKY0,None,KOG0977
25693,NV2t025986001.1,7.950000e-172,484.0,"KOG3656@1|root,KOG3656@2759|Eukaryota",45351.EDO34910|-,G-protein coupled receptor activity,-,-,-,-,-,Nvec,None,None,KOG3656


In [17]:
hvul = pd.read_table( 'Refs/HVAEP/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
hvul = hvul.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
hvul['Species'] = 'Hvul'
hvul['Metazoa'] = hvul.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
hvul['Bilateria'] = hvul.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
hvul['Eukaryota'] = hvul.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
hvul

/tmp/ipykernel_32011/943287169.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  hvul = pd.read_table( '../../Refs/HVAEP/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,HVAEP1.G000001,5.220000e-19,95.5,"2DANG@1|root,2TJZW@2759|Eukaryota,39GWZ@33154|...",7739.XP_002598531.1|-,ATP-grasp domain,-,-,-,-,-,Hvul,3CKRG,3DINH,2TJZW
1,HVAEP1.G000002,4.060000e-290,814.0,"COG0657@1|root,KOG4388@2759|Eukaryota,38DHI@33...",33208|Metazoa,hormone-sensitive lipase activity,LIPE,"GO:0000003,GO:0001676,GO:0003674,GO:0003676,GO...",3.1.1.79,ko:K07188,"Abhydrolase_3,HSL_N",Hvul,3BEW5,None,KOG4388
2,HVAEP1.G000004,4.880000e-23,110.0,"COG4637@1|root,COG4637@2|Bacteria,1TQ9X@1239|F...",186801|Clostridia,An automated process has identified a potentia...,-,-,-,-,"AAA-ATPase_like,HTH_3,PDDEXK_9",Hvul,None,None,None
3,HVAEP1.G000006,8.880000e-187,543.0,"KOG0667@1|root,KOG0667@2759|Eukaryota,38D9D@33...",33208|Metazoa,lens induction in camera-type eye,HIPK1,"GO:0000045,GO:0000122,GO:0001085,GO:0001102,GO...",2.7.11.1,ko:K08826,Pkinase,Hvul,3BCXE,None,KOG0667
4,HVAEP1.G000008,3.000000e-37,151.0,"COG4637@1|root,COG4637@2|Bacteria,1TQ9X@1239|F...",186801|Clostridia,An automated process has identified a potentia...,-,-,-,-,"AAA-ATPase_like,PDDEXK_9",Hvul,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21867,HVAEP15.G028909,7.570000e-23,102.0,"COG2801@1|root,KOG0017@2759|Eukaryota,38F2S@33...",33208|Metazoa,Gypsy retrotransposon,GIN1,-,-,-,"SCAN,THAP,rve,zf-CCHC,zf-H2C2",Hvul,3BI3C,3D3YU,KOG0017
21868,HVAEP15.G028910,2.050000e-213,595.0,"2CMTJ@1|root,2QRWJ@2759|Eukaryota,39SVI@33154|...",33208|Metazoa,TIR domain,-,-,-,-,TIR_2,Hvul,3BNJ9,None,2QRWJ
21869,HVAEP15.G028911,1.330000e-06,57.4,"2ANUN@1|root,2RZF8@2759|Eukaryota,3A1VU@33154|...",33208|Metazoa,-,-,-,-,-,MULE,Hvul,3BR7C,3D88E,2RZF8
21870,HVAEP15.G028913,1.100000e-224,623.0,"2CMTJ@1|root,2QRWJ@2759|Eukaryota,39SVI@33154|...",33208|Metazoa,TIR domain,-,-,-,-,TIR_2,Hvul,3BNJ9,None,2QRWJ


In [18]:
sbre = pd.read_table( 'Refs/go_Sbre_v1/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
sbre = sbre.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
sbre['Species'] = 'Sbre'
sbre['Metazoa'] = sbre.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
sbre['Bilateria'] = sbre.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
sbre['Eukaryota'] = sbre.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
sbre

/tmp/ipykernel_32011/934275504.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  sbre = pd.read_table( '../../Refs/go_Sbre_v1/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,go_Sbre_v1_10023_2_1.p1,2.570000e-66,213.0,"COG1104@1|root,KOG1549@2759|Eukaryota",2759|Eukaryota,cysteine desulfurase activity,NFS1,"GO:0000166,GO:0003674,GO:0003824,GO:0005488,GO...",2.8.1.7,ko:K04487,Aminotran_5,Sbre,None,None,KOG1549
1,go_Sbre_v1_1002_1_1.p1,5.010000e-44,151.0,"COG5272@1|root,KOG0004@2759|Eukaryota,38CAM@33...",33208|Metazoa,Ubiquitin-40S ribosomal protein,RPS27A,"GO:0005575,GO:0005622,GO:0005623,GO:0005634,GO...",-,ko:K02977,"Ribosomal_S27,ubiquitin",Sbre,3BDTF,3CUJQ,KOG0004
2,go_Sbre_v1_1003_2_1.p1,5.010000e-44,151.0,"COG5272@1|root,KOG0004@2759|Eukaryota,38CAM@33...",33208|Metazoa,Ubiquitin-40S ribosomal protein,RPS27A,"GO:0005575,GO:0005622,GO:0005623,GO:0005634,GO...",-,ko:K02977,"Ribosomal_S27,ubiquitin",Sbre,3BDTF,3CUJQ,KOG0004
3,go_Sbre_v1_10045_1_1.p1,1.980000e-24,106.0,"KOG1055@1|root,KOG1055@2759|Eukaryota,38DIA@33...",33208|Metazoa,gamma-aminobutyric acid type B receptor subunit,-,-,-,ko:K04615,"7tm_3,ANF_receptor",Sbre,3BB1F,3CSV7,KOG1055
4,go_Sbre_v1_10086_1_1.p2,2.040000e-86,274.0,"COG2216@1|root,COG2216@2|Bacteria,1MU7D@1224|P...",206389|Rhodocyclales,Part of the high-affinity ATP-driven potassium...,kdpB,-,3.6.3.12,ko:K01547,"E1-E2_ATPase,Hydrolase",Sbre,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17111,go_Sbre_v1_9966_1_1.p1,2.110000e-21,105.0,"KOG3646@1|root,KOG3646@2759|Eukaryota,38D2T@33...",33208|Metazoa,Neuronal acetylcholine receptor subunit alpha-...,nAChRa7,"GO:0003008,GO:0003674,GO:0005215,GO:0005216,GO...",-,"ko:K04809,ko:K05312","Neur_chan_LBD,Neur_chan_memb",Sbre,3BGXC,3CWAV,KOG3646
17112,go_Sbre_v1_9972_1_1.p1,2.550000e-135,425.0,"28KI0@1|root,2QSZB@2759|Eukaryota,38CI0@33154|...",33208|Metazoa,positive regulation of plasminogen activation,MFI2,"GO:0000041,GO:0003674,GO:0005488,GO:0005506,GO...",-,"ko:K06569,ko:K17283",Transferrin,Sbre,3B9KE,3CYY4,2QSZB
17113,go_Sbre_v1_9978_2_1.p1,3.530000e-14,75.5,"KOG1217@1|root,KOG1217@2759|Eukaryota",2759|Eukaryota,calcium ion binding,-,-,-,-,"Ephrin_rec_like,Sushi",Sbre,None,None,KOG1217
17114,go_Sbre_v1_9980_1_1.p1,4.060000e-09,60.1,"COG1770@1|root,COG1770@2|Bacteria,1MUED@1224|P...",204458|Caulobacterales,peptidase S9A prolyl oligopeptidase domain pro...,-,-,3.4.21.83,ko:K01354,"Peptidase_S9,Peptidase_S9_N",Sbre,None,None,None


In [19]:
pcro = pd.read_table( 'Refs/PROCRO/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )
pcro = pcro.drop( ['seed_ortholog','COG_category','KEGG_Pathway',
                   'KEGG_Module','KEGG_Reaction','KEGG_rclass',
                   'BRITE','KEGG_TC','CAZy','BiGG_Reaction'], axis=1 )
pcro['Species'] = 'Pcro'
pcro['Metazoa'] = pcro.eggNOG_OGs.apply( lambda x: parseOG( x, metazoa ) )
pcro['Bilateria'] = pcro.eggNOG_OGs.apply( lambda x: parseOG( x, bilateria ) )
pcro['Eukaryota'] = pcro.eggNOG_OGs.apply( lambda x: parseOG( x, eukaryota ) )
pcro

/tmp/ipykernel_32011/1296975997.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pcro = pd.read_table( '../../Refs/PROCRO/EGGNOG.emapper.annotations', skiprows=4, skipfooter=3 )


,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,Pcro_g11222.t1,4.700000e-69,256.0,"COG2801@1|root,KOG0017@2759|Eukaryota,37NRU@33...",35493|Streptophyta,3-hydroxyisobutyryl-CoA hydrolase activity,-,"GO:0000943,GO:0003674,GO:0003676,GO:0003723,GO...",-,-,"Chromo,RVP_2,RVT_1,Retrotrans_gag,rve",Pcro,None,None,KOG0017
1,Pcro_g11224.t1,6.710000e-66,233.0,"KOG3105@1|root,KOG3105@2759|Eukaryota,38DQS@33...",33208|Metazoa,DDE superfamily endonuclease,-,-,-,-,"DDE_1,HTH_psq",Pcro,3BGR7,3D3JB,KOG3105
2,Pcro_g11225.t1,4.610000e-12,71.6,"2CN11@1|root,2QT83@2759|Eukaryota,38D3X@33154|...",33208|Metazoa,substituted 1 base at 1 genomic stop codon,ZBED8,-,-,-,"DUF4371,Dimer_Tnp_hAT",Pcro,3BDRF,3CX54,2QT83
3,Pcro_g813.t1,2.030000e-15,91.3,"KOG4297@1|root,KOG4297@2759|Eukaryota,38GR9@33...",33208|Metazoa,carbohydrate binding,-,-,-,ko:K06560,"F5_F8_type_C,Laminin_G_3,Lectin_C,Methyltransf...",Pcro,3BBE1,3CTZ0,KOG4297
4,Pcro_g5862.t1,1.590000e-16,89.0,"COG5640@1|root,KOG3627@2759|Eukaryota,38CGF@33...",33208|Metazoa,serine-type endopeptidase activity. It is invo...,-,"GO:0003674,GO:0003824,GO:0004175,GO:0004252,GO...",3.4.21.49,ko:K20752,Trypsin,Pcro,3BBU3,3CSPP,KOG3627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31704,Pcro_g10869.t5,0.000000e+00,1460.0,"COG5599@1|root,KOG4228@2759|Eukaryota,38C1K@33...",33208|Metazoa,Fibronectin type 3 domain,PTPRD,"GO:0000003,GO:0000902,GO:0000904,GO:0001558,GO...",3.1.3.48,"ko:K05695,ko:K06777,ko:K06778","I-set,Ig_3,Y_phosphatase,fn3",Pcro,3BA83,3CXCZ,KOG4228
31705,Pcro_g10869.t6,4.880000e-62,223.0,"COG5599@1|root,KOG4228@2759|Eukaryota,38C1K@33...",33208|Metazoa,"Protein tyrosine phosphatase, receptor type, f, b",PTPRF,"GO:0000003,GO:0000902,GO:0000904,GO:0001558,GO...",3.1.3.48,"ko:K05695,ko:K06777","I-set,Ig_3,Y_phosphatase,fn3",Pcro,3BA83,3CXCZ,KOG4228
31706,Pcro_g10872.t1,3.480000e-28,123.0,"KOG1075@1|root,KOG1075@2759|Eukaryota,39XVS@33...",4751|Fungi,Reverse transcriptase (RNA-dependent DNA polym...,-,-,-,-,"Exo_endo_phos_2,RNase_H,RVT_1",Pcro,None,None,KOG1075
31707,Pcro_g5098.t1,5.120000e-07,56.6,"2D0D8@1|root,2SDRG@2759|Eukaryota,3ABMK@33154|...",33208|Metazoa,Alcohol dehydrogenase transcription factor Myb...,-,"GO:0002118,GO:0002121,GO:0007610,GO:0008150,GO...",-,-,MADF_DNA_bdg,Pcro,3BW03,3DH48,2SDRG


In [20]:
allGenes = pd.concat( (mlig,mcli,mhys,
                       smed,spol,
                       sman,sjap,
                       sbre,pcro,
                       plei,pdum,
                       clon,hmia,
                       nvec,hvul), axis=0, ignore_index=True )

allGenes

,#query,evalue,score,eggNOG_OGs,max_annot_lvl,Description,Preferred_name,GOs,EC,KEGG_ko,PFAMs,Species,Metazoa,Bilateria,Eukaryota
0,Mlig455_000001,6.190000e-16,89.7,"KOG2177@1|root,KOG2177@2759|Eukaryota,39W1B@33...",33208|Metazoa,interferon-beta production,TRIM56,"GO:0000003,GO:0000209,GO:0001816,GO:0001817,GO...",2.3.2.27,"ko:K11997,ko:K12026","NHL,zf-B_box,zf-C3HC4,zf-RING_UBOX",Mlig,3BIP6,3CRTC,KOG2177
1,Mlig455_000002,3.180000e-89,300.0,"KOG3512@1|root,KOG3512@2759|Eukaryota,39ZRD@33...",33208|Metazoa,Laminin N-terminal domain (domain VI),NTN3,"GO:0000768,GO:0000902,GO:0000904,GO:0003674,GO...",-,ko:K06844,"Laminin_EGF,Laminin_N,NTR",Mlig,3BNCU,3D4V1,KOG3512
2,Mlig455_000003,5.890000e-07,59.3,"28J4C@1|root,2QRGC@2759|Eukaryota,39AM6@33154|...",33208|Metazoa,Coiled-coil domain containing 150,CCDC150,-,-,ko:K15201,-,Mlig,3BGX4,3CVC0,2QRGC
3,Mlig455_000004,3.240000e-67,226.0,"KOG1655@1|root,KOG1655@2759|Eukaryota,38RIY@33...",33208|Metazoa,vacuolar transport,CHMP5,"GO:0000070,GO:0000278,GO:0000280,GO:0000815,GO...",-,ko:K12198,Snf7,Mlig,3BEAB,None,KOG1655
4,Mlig455_000005,6.060000e-31,134.0,"COG1597@1|root,KOG1116@2759|Eukaryota,38DRX@33...",33208|Metazoa,Sphingosine kinase 2,SPHK2,"GO:0000003,GO:0000323,GO:0001505,GO:0001568,GO...",2.7.1.91,ko:K04718,DAGK_cat,Mlig,3BDM3,3CZEF,KOG1116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340364,HVAEP15.G028909,7.570000e-23,102.0,"COG2801@1|root,KOG0017@2759|Eukaryota,38F2S@33...",33208|Metazoa,Gypsy retrotransposon,GIN1,-,-,-,"SCAN,THAP,rve,zf-CCHC,zf-H2C2",Hvul,3BI3C,3D3YU,KOG0017
340365,HVAEP15.G028910,2.050000e-213,595.0,"2CMTJ@1|root,2QRWJ@2759|Eukaryota,39SVI@33154|...",33208|Metazoa,TIR domain,-,-,-,-,TIR_2,Hvul,3BNJ9,None,2QRWJ
340366,HVAEP15.G028911,1.330000e-06,57.4,"2ANUN@1|root,2RZF8@2759|Eukaryota,3A1VU@33154|...",33208|Metazoa,-,-,-,-,-,MULE,Hvul,3BR7C,3D88E,2RZF8
340367,HVAEP15.G028913,1.100000e-224,623.0,"2CMTJ@1|root,2QRWJ@2759|Eukaryota,39SVI@33154|...",33208|Metazoa,TIR domain,-,-,-,-,TIR_2,Hvul,3BNJ9,None,2QRWJ


In [21]:
del mlig
del smed
del sman
del mcli
del mhys
del spol
del sjap
del plei
del pdum
del clon
del hmia
del nvec
del hvul
del sbre
del pcro

gc.collect()

72

In [22]:
orthos = pd.read_table( 'GenePhylogeny/OrthoFinderResults/Orthologues/Smed.tsv', index_col=0 )
orthos.loc[orthos.Species=='Pcro','Orthologs'] = [ ', '.join(['Pcro|Pcro_'+o.split('|')[-1] for o in entr.split(', ')]) \
                                                   for entr in orthos[orthos.Species=='Pcro'].Orthologs ]


orthos

,Species,Smed,Orthologs
Orthogroup,,,
OG0000000,Pdum,Smed|SMESG000048845.1,Pdum|comp223257_c0_seq6.p2
OG0000000,Mcli,Smed|SMESG000048845.1,"Mcli|Mcli002794.g.t1, Mcli|Mcli029842.g.t1"
OG0000000,Pcro,Smed|SMESG000048845.1,"Pcro|Pcro_g42493.t1, Pcro|Pcro_g25321.t1"
OG0000000,Mhys,Smed|SMESG000048845.1,"Mhys|Mhtx002007.g.t1, Mhys|Mhtx034101.g.t1, Mh..."
OG0000000,Hvul,"Smed|SMESG000032570.1, Smed|SMESG000078761.1, ...",Hvul|HVAEP2.G003309
...,...,...,...
OG0046390,Spol,Smed|SMESG000081840.1,Spol|dd_Spol_v4_66082_1_1.p1
OG0046391,Spol,Smed|SMESG000081851.1,Spol|dd_Spol_v4_13174_1_1.p1
OG0046392,Spol,Smed|SMESG000081862.1,Spol|dd_Spol_v4_13787_12_1.p1


In [23]:
def getGeneCluster( target, geneTable, level='Bilateria' ):
    targetOG = geneTable.loc[geneTable.iloc[:,0]==target,level].values[0]
    return geneTable[geneTable[level]==targetOG].iloc[:,0]

In [24]:
def getGenesFromOrthoFinder( orthoTable, geneTarget ):
    og = orthoTable.loc[orthoTable.iloc[:,1].str.contains(geneTarget),:]
    out = np.concatenate( [ g.split( ', ' ) for g in og.Orthologs ] )
    out = np.concatenate( ( out, og.iloc[0,1].split( ', ' ) ) )
    out = np.unique( [ '|'.join( g.split( '|' )[1:] ) for g in out ] )
    return out

In [25]:
spMap = { 'Mli': 'Mlig', 'Mcl': 'Mcli', 'Mht': 'Mhys',
          'SME': 'Smed', 'Smp': 'Sman', 'EWB': 'Sjap',
          'dd_': 'Spol', 'HMI': 'Hmia', 'clu': 'Clon',
          'NV2': 'Nvec', 'HVA': 'Hvul', 'Pri': 'Plei',
          'com': 'Pdum', 'go_': 'Sbre', 'Pcr': 'Pcro' }

In [35]:
# Function to perform first level of alignment and tree building 
# @param smedTarget: contig ID for the Smed gene of interest
# @param shortName: common gene name to be used for saving the results
# @param eggnogLevel: which EGGNOG orthology level to consider.
#   Options are Eukaryota, Metazoa, and Bilateria
# @param aligner: which protein aligner to use. Options are guidance, 
#   clustalo, and linsi (default)
# @param doClustering: whether to actually run alignment and tree building
# @param blacklist: list of genes to exclude from the phylogeny
# @param customGeneList: directly provided list of genes to align, rather
#   than checking the input resources for putative homologs
# @param extraGenes: list of additional genes to consider for the phylogeny
def doAnalysisFromSmed( smedTarget, shortName, eggnogLevel='Metazoa', aligner='guidance',
                        doClustering=True, blacklist=None, customGeneList=None, extraGenes=None ):
    # Load in the set of gene IDs we want to consider
    if customGeneList is not None:
        cGenes = np.array(customGeneList)
    else:
        # Get genes with the same OG tag as the planarian gene
        cGenes = np.array([])
        if smedTarget in allGenes.iloc[:,0].values:
            cGenes = getGeneCluster( smedTarget, allGenes, eggnogLevel ).values
        cGenes = np.sort(list(set(cGenes) |\
                         set(getGenesFromOrthoFinder( orthos, smedTarget ))))
    if extraGenes is not None:
        cGenes = np.sort(np.unique(np.concatenate((cGenes,extraGenes))))
    if blacklist is not None:
        cGenes = cGenes[~np.isin(cGenes,blacklist)]
    # Map each gene to its species, and write it out for GeneRax
    cSp = np.array([ spMap[g[:3]] for g in cGenes ])
    with open( 'GenePhylogeny/{0}_genemap.tr'.format(shortName), 'w' ) as f:
        f.writelines( [ ' '.join(s) + '\n' for s in zip(cGenes,cSp) ] )
    
    # Export the sequences of all of the genes
    outRecs = []
    for rec in SeqIO.parse( 'GenePhylogeny/all_proteins.aa', 'fasta' ):
        if rec.id in cGenes:
            # Write a simplified version of the record 
            # without the extra description
            outRecs.append( SeqIO.SeqRecord( rec.seq, rec.id, description='' ) )
    print( 'Writing sequences of {0} genes'.format(len(outRecs)) )
    print( cGenes )
    SeqIO.write( outRecs, 'GenePhylogeny/{0}.aa'.format(shortName), 'fasta' )
    
    famString = ''
    if doClustering:
        # Align the genes
        fname = 'GenePhylogeny/{0}_Results'.format(shortName)
        if not os.path.exists( fname ):
            os.mkdir( fname )
        print( 'Running alignment ...' )
        if aligner.lower() in ['clustalo', 'clustal']:
            # may need to updated clustalo path depending on your environment
            subprocess.run( 'clustalo -i GenePhylogeny/{0}.aa --threads=8 -o' + \
                            ' GenePhylogeny/{0}_Results/alignment.fa --force'\
                            .format(shortName), shell=True )
        elif aligner.lower() in ['guidance', 'guidance2']:
            # may need to update these paths depending on your environment
            pathToConda = '/home/gary/anaconda2/bin/conda'
            pathToGuidance = '/home/gary/anaconda2/envs/guidance' +\
                             '/opt/guidance/www/Guidance/guidance.pl'
            # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # This needs to be updated to point to the directory
            #   which contains this notebook
            # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            workingDir = '$JGWD/CrossSpeciesMappings/'
            subprocess.run( f'''conda init bash
                                conda run -n guidance \
                                perl {pathToGuidance} --seqFile GenePhylogeny/{shortName}.aa --msaProgram MAFFT \
                                --seqType aa --outDir {workingDir}/GenePhylogeny/{shortName}_Results \
                                --seqCutoff 0.2 --colCutoff 0.9 --MSA_Param \"\\-\\-localpair\" \
                                --proc_num 8 --outOrder as_input''',
                            executable='/bin/bash', shell=True )
            subprocess.run( 'mv GenePhylogeny/{0}_Results/MSA.MAFFT.Without_low_SP_Col.With_Names'\
                               .format(shortName) +
                            ' GenePhylogeny/{0}_Results/alignment.fa'.format(shortName), shell=True )
        else:
            subprocess.run( 'linsi --thread 8 GenePhylogeny/{0}.aa > GenePhylogeny/{0}_Results/alignment.fa'\
                            .format(shortName), shell=True )
        # Fit the initial tree model
        print( 'Selecting tree model ...' )
        subprocess.run( '~/iqtree-2.3.4/bin/iqtree2' + \
                        ' -s GenePhylogeny/{0}_Results/alignment.fa' + \
                        ' -pre GenePhylogeny/{0}_Results/iqt'.format(shortName) + \
                        ' -nt 8 -seed 0 -bb 1000 -alrt 1000 -m MFP+G -mset raxml -redo', shell=True )
        # Update the family string for GeneRax
        famString += '- {0}\n'.format(shortName)
        famString += 'starting_gene_tree = GenePhylogeny/{0}_Results/iqt.contree\n'.format(shortName)
        famString += 'alignment = GenePhylogeny/{0}_Results/alignment.fa\n'.format(shortName)
        famString += 'mapping = GenePhylogeny/{0}_genemap.tr\n'.format(shortName)
        modelStr = ''
        with gzip.open( 'GenePhylogeny/{0}_Results/iqt.model.gz'.format(shortName) ) as f:
            for line in f.readlines():
                if line.decode('utf-8').startswith( 'best_model_AIC:' ):
                    modelStr = line.decode('utf-8').split(':')[1].strip()
                    break
        famString += 'subst_model = {0}\n'.format(modelStr)
        
    print( 'DONE' )
    return famString

In [37]:
doClustering = True
famString = '[FAMILIES]\n'

In [48]:
# ETV (Neural)

famString += doAnalysisFromSmed( 'SMESG000008885.1', 'ETV',
                                 eggnogLevel='Metazoa', 
                                 aligner='clustalo', 
                                 doClustering=doClustering )

Writing sequences of 21 genes
['EWB00_003444' 'HMIM022324-PA' 'HVAEP11.G020207' 'Mcli003206.g.t2'
 'Mcli038571.g.t1' 'Mhtx013194.g.t1' 'Mhtx015632.g.t1' 'Mlig455_009762'
 'Mlig455_054310' 'Mlig455_070121' 'NV2t012676002.1' 'Pcro_g32949.t1'
 'PrileiEVm007695t1.p1' 'SMESG000008885.1' 'SMESG000033317.1'
 'Smp_342280.1' 'comp135579_c0_seq1.p1' 'dd_Spol_v4_16811_1_1.p1'
 'dd_Spol_v4_43403_1_1.p1' 'go_Sbre_v1_44189_2_1.p1'
 'go_Sbre_v1_46450_1_1.p1']
Running alignment ...
Selecting tree model ...
IQ-TREE multicore version 2.3.4 COVID-edition for Linux x86 64-bit built Apr 26 2024
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung, Olga Chernomor,
Heiko Schmidt, Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    gary-desktop (AVX2, FMA3, 62 GB RAM)
Command: /home/gary/iqtree-2.3.4/bin/iqtree2 -s ETV_Results/alignment.fa -pre ETV_Results/iqt -nt 8 -seed 0 -bb 1000 -alrt 1000 -m MFP+G -mset raxml -redo
Seed:    0 (Using SPRNG - Scalable Parallel Random Number Gen

In [ ]:
# POU4

famString += doAnalysisFromSmed( 'SMESG000030759.1', 'POU4',
                                 eggnogLevel='Metazoa', 
                                 aligner='clustalo', 
                                 doClustering=doClustering )

In [ ]:
# PAX6L

famString += doAnalysisFromSmed( 'SMESG000004009.1', 'PAX6L',
                                 eggnogLevel='Metazoa', 
                                 aligner='clustalo', 
                                 doClustering=doClustering,
                                 extraGenes=['Mlig455_036182','Mlig455_005549'])

In [ ]:
# Fixing a weird idionsyncracy with the way IQTree
#   and GeneRax name their substitution models
famString = famString.replace( 'JTTDCMut', 'JTT-DCMut' )

In [ ]:
# Write out the families file
with open( 'GenePhylogeny/generax_families.txt', 'w' ) as f:
    f.write( famString )

In [ ]:
# Run GeneRax
# Will need to update the conda environment to
#   point to one with GeneRax installed
# This also includes some other genes that were 
#   run in parallel, but not relevant for this
#   analysis
subprocess.run( f'''conda init bash
                    conda run -n eggnog-jg \
                    generax -f GenePhylogeny/generax_families.txt \
                    -s GenePhylogeny/speciesTree.nw \
                    -p GenePhylogeny/geneRaxOut20240604 --strategy SPR''',
                executable='/bin/bash', shell=True )

Individual gene trees from the path GenePhylogeny/geneRaxOut/results/GENE_NAME/geneTree.newick were uploaded to the website phylo.io for visualization